In [ ]:
import os
import os.path as op
import pandas as pd
from scipy.spatial.distance import cdist
import numpy as np
from matplotlib import pyplot as plt
import pickle
from nilearn.image import load_img, new_img_like
import nibabel as nib
from nilearn import datasets
from nilearn.maskers import NiftiMasker
from nilearn.masking import apply_mask
from nilearn.plotting import plot_roi
from frechetdist import frdist
from scipy.ndimage import gaussian_filter
from scipy.stats import spearmanr
from scipy.stats import ttest_ind, ttest_rel
import glob
from scipy.signal import decimate
import similaritymeasures as sm
from joblib import Parallel, delayed

In [ ]:
import sys
sys.path.append('.')
import eyenalysis as mathot

In [ ]:
dirname = "/Users/radhanila/Desktop/Radha_Postdoc_OVGU/Nico_eye/"

In [ ]:
#### print all rows
pd.set_option('display.max_rows', None)

In [ ]:

def get_eye_rdm_subj_wise(dist_type):
    files = sorted(glob.glob(op.join(dirname, 'eye_data', 'sub-**_ses-movie_task-movie.npz')))
    outpath = op.join(dirname, 'output_subjwise')

    def fill_na_vals(df):
        df['x'] = df.x.interpolate(method='slinear')
        df['x'] = df.x.ffill()
        df['x'] = df.x.bfill()

        df['y'] = df.y.interpolate(method='slinear')
        df['y'] = df.y.ffill()
        df['y'] = df.y.bfill()

        return(df)

    def calc_dist_arr(fix_list, dist_arr, dist_type):

        def get_frechet_dist(i, j, i_ind, j_ind):
            if i_ind != j_ind:
                dist = sm.frechet_dist(i, j)
                dist_arr[i_ind,j_ind]=dist

        if dist_type == 'frechet':
            print('here')
            # for i_ind, i in enumerate(fix_list):
            #     for j_ind, j in enumerate(fix_list):
            #         if (i_ind != j_ind):
            #             dist = sm.frechet_dist(i, j)
            #             dist_arr[i_ind,j_ind]=dist

            Parallel(n_jobs=2)(delayed(get_frechet_dist)(i, j, i_ind, j_ind) for j_ind, j in enumerate(fix_list) for i_ind, i in enumerate(fix_list))
                
                        
        elif dist_type == 'mathot':
            for i_ind, i in enumerate(fix_list):
                for j_ind, j in enumerate(fix_list):
                    if (i_ind != j_ind):
                        dist = mathot.sp_dist(i, j, norm=False)
                        dist_arr[i_ind,j_ind]=dist
        else:
            for i_ind, i in enumerate(fix_list):
                for j_ind, j in enumerate(fix_list):
                    if (i_ind != j_ind):
                        # print(i)
                        # print(j)
                        dist = np.mean(cdist(i, j, metric=dist_type))
                        dist_arr[i_ind,j_ind]=dist
                    
        return(dist_arr)
    
    def plot_rdm(dist_arr, outpath, chunk_names):
        plt.imshow(dist_arr, origin='lower')
        plt.xticks([0, len(chunk_names)-1],[chunk_names[0], chunk_names[-1]])
        plt.yticks([0, len(chunk_names)-1],[chunk_names[0], chunk_names[-1]])
        plt.colorbar()
        plt.savefig(op.join(outpath, 'fig', dist_type, f"{subj}_ses-movie_task-movie_run-0{run}_rdm-{dist_type}.png"))
        plt.close()

    for eye_file in files:
        ### Remove sub-05
        subj = op.basename(eye_file)[:6]
        eye_arr = np.load(eye_file)
        keys = eye_arr.files
        
        for run in range(1,9):
            run_inds = np.where(eye_arr['sa.movie_run']==run)[0]
            print(f"Processing subj {subj} run {run} for dist_type {dist_type}")
            events = pd.read_csv(op.join(dirname, 'eye_data', f'ses-movie_task-movie_run-0{run}_events.tsv'), sep='\t')
            onsets = events.onset.values.tolist()
            onsets = [o-1 for o in onsets] ### Uncorrect slice time correction
            chunk_names = events.trial_type.values.tolist()

            samples = eye_arr['samples'][run_inds]
            samples_df = pd.DataFrame(samples, columns = ['x', 'y', 'pup'])
            # xvals = samples_df.x.values.tolist() ### No interpolation
            # yvals = samples_df.y.values.tolist()
            df = fill_na_vals(samples_df) ### Interpolation
            xvals = df.x.values.tolist()
            yvals = df.y.values.tolist()
        
            offset = 4000
            xy_list = []
            final_chunk_list = []
            
            # cnt = 0
            for st_ind, st in enumerate(onsets):
                chunkx = xvals[int(st*1000): int(st*1000) + offset]
                chunky = yvals[int(st*1000): int(st*1000) + offset]
                
                chunkxy = np.array(list(zip(chunkx, chunky)))
                chunkxy = chunkxy[::4] ## downsample
                
                if chunkxy.size:
                    xy_list.append(chunkxy)
                    final_chunk_list.append(chunk_names[st_ind])
            
            arr_size = len(xy_list)
            dist_arr = np.zeros((arr_size, arr_size))
            
            dist_arr = calc_dist_arr(xy_list, dist_arr, dist_type)

            np.save(op.join(outpath, 'rdm', dist_type, f"{subj}_ses-movie_task-movie_run-0{run}_rdm-{dist_type}.npy"), dist_arr)
            
            plot_rdm(dist_arr, outpath, dist_type, final_chunk_list)
            break
        break            

# for dist_type in ['euclidean', 'mathot', 'cosine', 'mahalanobis', 'frechet']:
# for dist_type in ['frechet']: 
# for dist_type in ['sqeuclidean']:
# for dist_type in ['euclidean', 'sqeuclidean', 'mathot', 'cosine']:
for dist_type in ['frechet']: 
    print(f"Processing distance {dist_type}")
    get_eye_rdm_subj_wise(dist_type)
    # break

In [ ]:
######## Scene vs scene RDM

def get_eye_rdm_chunkwise(dist_type):
    files = sorted(glob.glob(op.join(dirname, 'eye_data', 'sub-**_ses-movie_task-movie.npz')))
    outpath = op.join(dirname, 'output_chunkwise')

    def fill_na_vals(df):
        df['x'] = df.x.interpolate(method='slinear')
        df['x'] = df.x.ffill()
        df['x'] = df.x.bfill()

        df['y'] = df.y.interpolate(method='slinear')
        df['y'] = df.y.ffill()
        df['y'] = df.y.bfill()

        return(df)

    def calc_dist_arr(fix_list, dist_arr, dist_type):
        if dist_type == 'frechet':
            for i_ind, i in enumerate(fix_list):
                for j_ind, j in enumerate(fix_list):
                    if (i_ind != j_ind):
                        dist = frdist(i, j)
                        dist_arr[i_ind,j_ind]=dist
        elif dist_type == 'mathot':
            for i_ind, i in enumerate(fix_list):
                for j_ind, j in enumerate(fix_list):
                    if (i_ind != j_ind):
                        dist = mathot.sp_dist(i, j, norm=False)
                        dist_arr[i_ind,j_ind]=dist
        else:
            for i_ind, i in enumerate(fix_list):
                for j_ind, j in enumerate(fix_list):
                    if (i_ind != j_ind):
                        # print(i)
                        # print(j)
                        # print(i, j)
                        dist = np.mean(cdist(i, j, metric=dist_type))
                        dist_arr[i_ind,j_ind]=dist
                    
        return(dist_arr)
    
    def plot_rdm(dist_arr, outpath, subj_list, chunk_name):
        plt.imshow(dist_arr, origin='lower')
        plt.xticks([0, len(subj_list)-1],[subj_list[0], subj_list[-1]])
        plt.yticks([0, len(subj_list)-1],[subj_list[0], subj_list[-1]])
        plt.colorbar()
        plt.savefig(op.join(outpath, 'fig', f"ses-movie_task-movie_run-0{run}_chunk-{chunk_name}_rdm-{dist_type}.png"))
        plt.close()

    offset = 4000
    for run in range(8,9):
        events = pd.read_csv(op.join(dirname, 'eye_data', f'ses-movie_task-movie_run-0{run}_events.tsv'), sep='\t')
        onsets = events.onset.values.tolist()
        onsets = [o-1 for o in onsets] ### Uncorrect slice time correction
        chunk_names = events.trial_type.values.tolist()

        print(f"Processing run {run} for dist_type {dist_type}")

        print(len(onsets), len(chunk_names))

        cnt = 0
        for st_ind, st in enumerate(onsets):
            xy_list = []
            subj_list = []
            # print(chunk_names[st_ind])
            # break
            for eye_file in files:
                ### Remove sub-05
                subj = op.basename(eye_file)[:6]

                if subj != 'sub-05':
                    eye_arr = np.load(eye_file)
                    keys = eye_arr.files

                    ### Get run samples for subj
                    run_inds = np.where(eye_arr['sa.movie_run']==run)[0]
                    # print(f"Processing subj {subj} run {run} for dist_type {dist_type}")
                
                    samples = eye_arr['samples'][run_inds]
                    samples_df = pd.DataFrame(samples, columns = ['x', 'y', 'pup'])
                    # xvals = samples_df.x.values.tolist() ### No interpolation
                    # yvals = samples_df.y.values.tolist()
                    df = fill_na_vals(samples_df) ### Interpolation 
                    xvals = df.x.values.tolist() 
                    yvals = df.y.values.tolist()
                    
                    chunkx = xvals[int(st*1000): int(st*1000) + offset]
                    chunky = yvals[int(st*1000): int(st*1000) + offset]
                    
                    chunkxy = np.array(list(zip(chunkx, chunky)))
                    chunkxy = chunkxy[::2] ## downsample
                    
                    if chunkxy.size:
                        xy_list.append(chunkxy)
                        subj_list.append(subj)
        
            arr_size = len(xy_list)
            dist_arr = np.zeros((arr_size, arr_size))
            # print(dist_arr.shape)
        
            dist_arr = calc_dist_arr(xy_list, dist_arr, dist_type)

            np.save(op.join(outpath, 'rdm', f"ses-movie_task-movie_run-0{run}_{chunk_names[st_ind]}_rdm-{dist_type}.npy"), dist_arr)
        
            # print(subj_list)
            plot_rdm(dist_arr, outpath, subj_list, chunk_names[st_ind])
            # break
        # break            

# for dist_type in ['euclidean', 'mathot', 'cosine', 'mahalanobis', 'frechet']:
# for dist_type in ['frechet']: 
# for dist_type in ['sqeuclidean']:
# for dist_type in ['euclidean', 'sqeuclidean', 'mathot', 'cosine']:
# for dist_type in ['mathot', 'cosine']:
for dist_type in ['frechet']:
    print(f"Processing distance {dist_type}")
    get_eye_rdm_chunkwise(dist_type)
    # break


In [ ]:

def get_fix_data_for_chunks():

    asc_dir = '/Users/radhanila/Desktop/Radha_Postdoc_OVGU/Nico_eye/asc_files/'
    eye_files = sorted(glob.glob(op.join(asc_dir, f"sub-**_ses-movie_task-movie_run-*_eyelinkraw.asc")))

    for eye_file in eye_files:
        parts = op.basename(eye_file).split('_')
        sub = parts[0]
        run = int(parts[3].split('-')[1])

        print(f"Processing {sub} run-{run}")

        events = pd.read_csv(op.join(dirname, 'eye_data', f'ses-movie_task-movie_run-0{run}_events.tsv'), sep='\t')
        eye_file = op.join(dirname, 'eye_data', f'{sub}_ses-movie_task-movie.npz')
        
        eye_arr = np.load(eye_file)
        run_inds = np.where(eye_arr['sa.movie_run']==run)[0]

        samples = eye_arr['samples'][run_inds]
        samples_df = pd.DataFrame(samples, columns = ['x', 'y', 'pup'])

        frames = eye_arr['sa.movie_run_frame'][run_inds]
        
        onsets = events.onset.values.tolist()
        onsets = events.onset.values.tolist()
        onsets = [o-1 for o in onsets] ### Uncorrect slice time correction

        chunk_frames = []
        for each_onset in onsets:
            ind = int(each_onset*1000)
            chunk_frames.append(frames[ind])

        
        ###### Map chunk frames to asc frames
        frame_str = 'FIDX'

        frame_fix = []
        frame = -1
        frame_flag = False
        all_chunk_fix = []

        #### Get fixations in each chunk
        with open(op.join(asc_dir, f'{sub}_ses-movie_task-movie_run-{run}_eyelinkraw.asc'), 'r') as in_file:
            lines = in_file.readlines()
            recording = False
            for line in lines:
                if 'START' in line:
                    recording = True
                    # print(line)
                if 'END' in line:
                    recording = False
                    # print(line)
                if recording and frame_str in line:
                    frame = int(line.split()[-1])-1
                    if frame in chunk_frames:
                        # print('frame', frame)
                        frame_flag = True
                        if frame != chunk_frames[0]: ## If not the first frame
                            # print('here')
                            all_chunk_fix.append(chunk_fix)
                            # break
                        frame_start = int(line.split()[1])
                        frame_end = frame_start + 4000
                        chunk_fix = []
                if frame_flag and 'EFIX' in line:
                    line_list = line.split()
                    # print(line_list)
                    if int(line_list[2]) > frame_start and int(line_list[3]) < frame_end:
                        dur = int(line_list[3]) - int(line_list[2]) ### Duration has weird, constant large value in front of a variable, which also seems wrong
                        chunk_fix.append([float(line_list[5]), float(line_list[6]), dur, float(line_list[7])])
                    else:
                        frame_flag = False
            all_chunk_fix.append(chunk_fix)
                    
        with open(op.join(dirname, 'fix_data', f'{sub}_ses-movie_task-movie_run-{run}_fixations.pkl'), 'wb') as f:
            pickle.dump(all_chunk_fix, f)

        with open(op.join(dirname, 'fix_data', f'{sub}_ses-movie_task-movie_run-{run}_fixations.pkl'), 'rb') as f:
            all_chunk_fix_loaded = pickle.load(f)

        print(f"Length of onsets {len(onsets)}, chunks obtained for {len(all_chunk_fix_loaded)}")

get_fix_data_for_chunks()


In [ ]:
##### Get RDMs for fix data
def get_fix_rdm_chunkwise(dist_type):
    
    def calc_dist_arr(fix_list, dist_arr, dist_type):
        if dist_type == 'frechet':
            for i_ind, i in enumerate(fix_list):
                for j_ind, j in enumerate(fix_list):
                    if (i_ind != j_ind):
                        if len(i) < len(j):
                            dist = frdist(i, j[:len(i)])
                        else:
                            dist = frdist(i[:len(j)], j)
                        dist_arr[i_ind,j_ind]=dist
        elif dist_type == 'mathot':
            for i_ind, i in enumerate(fix_list):
                for j_ind, j in enumerate(fix_list):
                    if (i_ind != j_ind):
                        dist = mathot.sp_dist(i, j, norm=False)
                        dist_arr[i_ind,j_ind]=dist
                    
        return(dist_arr)
    
    def plot_rdm(dist_arr, outpath, subj_list, chunk_name):
        plt.imshow(dist_arr, origin='lower')
        plt.xticks([0, len(subj_list)-1],[subj_list[0], subj_list[-1]])
        plt.yticks([0, len(subj_list)-1],[subj_list[0], subj_list[-1]])
        plt.colorbar()
        plt.savefig(op.join(outpath, 'fig', f"fix_{dist_type}", f"ses-movie_task-movie_run-0{run}_chunk-{chunk_name}_rdm-{dist_type}_fix.png"))
        plt.close()

    files = sorted(glob.glob(op.join(dirname, 'eye_data', 'sub-**_ses-movie_task-movie.npz')))
    subj_list = [op.basename(x).split('_')[0] for x in files]
    print(subj_list)

    outpath = op.join(dirname, 'output_chunkwise')
    
    offset = 4000
    for run in range(1,9):
        events = pd.read_csv(op.join(dirname, 'eye_data', f'ses-movie_task-movie_run-0{run}_events.tsv'), sep='\t')
        onsets = events.onset.values.tolist()
        onsets = [o-1 for o in onsets] ### Uncorrect slice time correction
        chunk_names = events.trial_type.values.tolist()

        print(f"Processing run {run} for dist_type {dist_type}")

        print(len(onsets), len(chunk_names))

        cnt = 0
        for st_ind, st in enumerate(onsets):
            fix_list = []

            sel_subj_list = []
            for subj in subj_list:
                with open(op.join(dirname, 'fix_data', f'{subj}_ses-movie_task-movie_run-{run}_fixations.pkl'), 'rb') as f:
                    all_chunk_fix_loaded = pickle.load(f)
                subj_chunk_fix = all_chunk_fix_loaded[st_ind]
                if subj_chunk_fix:
                    sel_subj_list.append(subj)
                    fix_list.append(subj_chunk_fix)
        
            arr_size = len(fix_list)
            dist_arr = np.zeros((arr_size, arr_size))
            # print(dist_arr.shape)
        
            dist_arr = calc_dist_arr(fix_list, dist_arr, dist_type)

            np.save(op.join(outpath, 'rdm', f"fix_{dist_type}", f"ses-movie_task-movie_run-0{run}_{chunk_names[st_ind]}_rdm-{dist_type}_fix.npy"), dist_arr)
        
            print(sel_subj_list)
            plot_rdm(dist_arr, outpath, sel_subj_list, chunk_names[st_ind])
            # break
        # break

dist_type = 'mathot'
get_fix_rdm_chunkwise(dist_type)

In [ ]:
a = [1,2,3]
b=[4,5,6,7]
b[:len(a)]
